# Replacing Covid Tracking Project - Simplified Workflow

>NOTE: ~~See  final workflow in the final script: `data_acquisition.py`~~ **as of 08/12, the development of the data acquisition code has moved back to "`Exploring Options for Replacing Covid Tracking Project Data_08-2021.ipynb`"**

- 08/09/21

- This is a companion notebook to `Exploring Options for Replacing Covid Tracking Project Data_08-2021`
- The goal is to only save the final pieces of code required to produce the dataset, and to exclude testing things out and info displays

## Summary

- Download kaggle data for the deaths and cases counts
- Download Socrata HHS data for hospital info
- Get all as daily frequency and with State Abbrevs before merging

In [1]:
# !pip install -U fsds
from fsds.imports import *

import os,zipfile,json,joblib
pd.set_option('display.max_columns',0)

fsds v0.3.2 loaded.  Read the docs: https://fs-ds.readthedocs.io/en/latest/ 


Handle,Package,Description
dp,IPython.display,Display modules with helpful display and clearing commands.
fs,fsds,Custom data science bootcamp student package
mpl,matplotlib,Matplotlib's base OOP module with formatting artists
plt,matplotlib.pyplot,Matplotlib's matlab-like plotting module
np,numpy,scientific computing with Python
pd,pandas,High performance data structures and tools
sns,seaborn,High-level data visualization library based on matplotlib


In [2]:
## Appending folder with functions
%load_ext autoreload 
%autoreload 2
# import sys
# sys.path.append('.')

import functions as fn
# help(fn)

# Data

## Specifying File Destinations

In [3]:
## Specifying data storage folders
fpath_raw = r"data_raw"
fpath_clean = r"data/"
[os.makedirs(fpath,exist_ok=True) for fpath in [fpath_clean,fpath_raw]];

## Covid-19 Data From Johns Hopkins University

- https://www.kaggle.com/antgoldbloom/covid19-data-from-john-hopkins-university
- Comes with CONVENIENT_ files and RAW_ files.

In [4]:
## Download kaggle jhu data and make zipfile object
!kaggle datasets download -p "{fpath_raw}" -d antgoldbloom/covid19-data-from-john-hopkins-university

jhu_data_zip = zipfile.ZipFile(os.path.join(fpath_raw,'covid19-data-from-john-hopkins-university.zip'))
jhu_data_zip.namelist()

covid19-data-from-john-hopkins-university.zip: Skipping, found more recently modified local copy (use --force to force download)


['CONVENIENT_global_confirmed_cases.csv',
 'CONVENIENT_global_deaths.csv',
 'CONVENIENT_global_metadata.csv',
 'CONVENIENT_us_confirmed_cases.csv',
 'CONVENIENT_us_deaths.csv',
 'CONVENIENT_us_metadata.csv',
 'RAW_global_confirmed_cases.csv',
 'RAW_global_deaths.csv',
 'RAW_us_confirmed_cases.csv',
 'RAW_us_deaths.csv']

In [5]:
## Getting State Abbrevs
state_abbrevs = pd.read_csv('Reference Data/united_states_abbreviations.csv')

## Making dicts of Name:Abbrev and Abbrev:Name
state_to_abbrevs_map = dict(zip(state_abbrevs['State'],state_abbrevs['Abbreviation']))
abbrev_to_state_map = dict(zip(state_abbrevs['Abbreviation'],state_abbrevs['State']))
# state_to_abbrevs_map

### prep `df_metadata`

In [6]:
# metadata
file = 'CONVENIENT_us_metadata.csv'
jhu_data_zip.extract(file,path=fpath_raw)
df_metadata = pd.read_csv(os.path.join(fpath_raw,file))

## Adding State Abbrevas to kaggle metadata
df_metadata.insert(1,'State_Code',df_metadata['Province_State'].map(state_to_abbrevs_map))
print(df_metadata.isna().sum())

## Dropping us territories
df_metadata.dropna(subset=['State_Code'], inplace=True)

## Saving county info
df_metadata.to_csv(os.path.join(fpath_clean,"us_metadata_counties.csv"),index=False)
df_metadata

Province_State    0
State_Code        6
Admin2            6
Population        0
Lat               0
Long              0
dtype: int64


,Province_State,State_Code,Admin2,Population,Lat,Long
0,Alabama,AL,Autauga,55869,32.539527,-86.644082
1,Alabama,AL,Baldwin,223234,30.727750,-87.722071
2,Alabama,AL,Barbour,24686,31.868263,-85.387129
3,Alabama,AL,Bibb,22394,32.996421,-87.125115
4,Alabama,AL,Blount,57826,33.982109,-86.567906
...,...,...,...,...,...,...
3337,Wyoming,WY,Teton,23464,43.935225,-110.589080
3338,Wyoming,WY,Uinta,20226,41.287818,-110.547578
3339,Wyoming,WY,Unassigned,0,0.000000,0.000000
3340,Wyoming,WY,Washakie,7805,43.904516,-107.680187


In [7]:
## Saving a states-only version with aggregated populations and mean lat/long
df_state_metadata = df_metadata.groupby('Province_State',as_index=False).agg({'Population':'sum',
                                               "Lat":'mean',"Long":"mean"})
df_state_metadata.insert(1,'State_Code',df_state_metadata['Province_State'].map(state_to_abbrevs_map))
df_state_metadata.to_csv(os.path.join(fpath_clean,"us_metadata_states.csv"),index=False)
df_state_metadata

,Province_State,State_Code,Population,Lat,Long
0,Alabama,AL,4903185,31.931113,-84.196785
1,Alaska,AK,740995,56.628273,-139.571540
2,Arizona,AZ,7278717,29.714033,-98.349911
3,Arkansas,AR,3017804,34.005087,-90.033096
4,California,CA,39512223,36.582496,-116.704308
5,Colorado,CO,5758736,37.755612,-102.289687
6,Connecticut,CT,3565287,33.290944,-58.125464
7,Delaware,DE,973764,23.465566,-45.319942
8,District of Columbia,DC,705749,12.968059,-25.672187
9,Florida,FL,21477737,28.101892,-80.303621


In [8]:
## Making and saving remapping dicts
import joblib

state_to_abbrevs_meta = dict(zip(df_state_metadata['Province_State'],df_state_metadata['State_Code']))
abbrev_to_state_meta = dict(zip(df_state_metadata['State_Code'],df_state_metadata['Province_State']))

joblib.dump(state_to_abbrevs_meta, os.path.join(fpath_clean,'state_names_to_codes_map.joblib'))
joblib.dump(abbrev_to_state_meta, os.path.join(fpath_clean,'state_codes_to_names_map.joblib'))

## save mapper fo state to code for function
mapper_path = os.path.join(fpath_clean,'state_names_to_codes_map.joblib')
mapper_path

'data/state_names_to_codes_map.joblib'

In [9]:
def load_raw_ts_file(jhu_data_zip, file = 'RAW_us_confirmed_cases.csv',
                     mapper_path='data/state_names_to_codes_map.joblib',
                    verbose=True):
    
    if verbose: 
        print(f"Loading data from {file}")
    state_to_abbrevs_meta = joblib.load(mapper_path)
    
    ## Extract and load csv
    jhu_data_zip.extract(file)
    data = pd.read_csv(file)
    
    ## Drop states not included in metadata
    data.insert(1,'State_Code',data['Province_State'].map(state_to_abbrevs_meta))
    data.dropna(subset=['State_Code'],inplace=True)
    return data



def melt_df_to_ts(df_cases,value_name, var_name='Date',
                  multi_index_cols=['State_Code','Date'],
                  id_cols = ['Province_State',"State_Code",'Admin2'],
                  cols_to_drop=['iso2','iso3','code3','UID','Country_Region',
                                'Combined_Key','Lat','Long_','FIPS']):
    
#     value_cols = [c for c in df_cases.columns if c not in [*cols_to_drop,*id_cols]]
    
    ## Remove any cols not in the actual dataframe
    id_cols = [c for c in id_cols if c in df_cases.columns] 
    cols_to_drop = [c for c in cols_to_drop if c in df_cases.columns] 
    
    ## CHECKING FOR NON-DATE COLS TO REMOVE
    value_cols = [c for c in df_cases.columns if c not in [*id_cols,*cols_to_drop]]
    value_cols = list(filter(lambda x: len(x.split('/'))>1,value_cols))
    
    
    df_cases_ts = pd.melt(df_cases, 
                          id_vars=id_cols, value_vars=value_cols,
                          var_name=var_name, value_name=value_name)
    
    df_cases_ts['Date'] = pd.to_datetime(df_cases_ts['Date'])
    df_cases_ts = df_cases_ts.set_index(multi_index_cols).sort_index()
    return df_cases_ts

In [10]:
## Prep ` df_cases_ts`
df_cases = load_raw_ts_file(jhu_data_zip, file = 'RAW_us_confirmed_cases.csv',)
df_cases_ts = melt_df_to_ts(df_cases,'Cases')
df_cases_ts

Loading data from RAW_us_confirmed_cases.csv


Province_State          Admin2  Cases
State_Code Date                                            
AK         2020-01-22         Alaska  Aleutians East      0
           2020-01-22         Alaska  Aleutians West      0
           2020-01-22         Alaska       Anchorage      0
           2020-01-22         Alaska          Bethel      0
           2020-01-22         Alaska     Bristol Bay      0
...                              ...             ...    ...
WY         2021-08-12        Wyoming           Teton   4044
           2021-08-12        Wyoming           Uinta   2559
           2021-08-12        Wyoming      Unassigned      0
           2021-08-12        Wyoming        Washakie    968
           2021-08-12        Wyoming          Weston    701

[1898184 rows x 3 columns]

In [11]:
## Prep df_deaths_ts
df_deaths = load_raw_ts_file(jhu_data_zip,file = 'RAW_us_deaths.csv')
df_deaths_ts = melt_df_to_ts(df_deaths,'Deaths')
df_deaths_ts

Loading data from RAW_us_deaths.csv


Province_State          Admin2  Deaths
State_Code Date                                             
AK         2020-01-22         Alaska  Aleutians East       0
           2020-01-22         Alaska  Aleutians West       0
           2020-01-22         Alaska       Anchorage       0
           2020-01-22         Alaska          Bethel       0
           2020-01-22         Alaska     Bristol Bay       0
...                              ...             ...     ...
WY         2021-08-12        Wyoming           Teton      11
           2021-08-12        Wyoming           Uinta      14
           2021-08-12        Wyoming      Unassigned       0
           2021-08-12        Wyoming        Washakie      26
           2021-08-12        Wyoming          Weston       6

[1898184 rows x 3 columns]

In [12]:
pd.concat([df_cases_ts,df_deaths_ts],axis=1)

Province_State          Admin2  Cases Province_State          Admin2  Deaths
State_Code Date                                                                                   
AK         2020-01-22         Alaska  Aleutians East      0         Alaska  Aleutians East       0
           2020-01-22         Alaska  Aleutians West      0         Alaska  Aleutians West       0
           2020-01-22         Alaska       Anchorage      0         Alaska       Anchorage       0
           2020-01-22         Alaska          Bethel      0         Alaska          Bethel       0
           2020-01-22         Alaska     Bristol Bay      0         Alaska     Bristol Bay       0
...                              ...             ...    ...            ...             ...     ...
WY         2021-08-12        Wyoming           Teton   4044        Wyoming           Teton      11
           2021-08-12        Wyoming           Uinta   2559        Wyoming           Uinta      14
           2021-08-12        Wyoming      Unassigned      0        Wyoming      Unassigned       0
           2021-08-12        Wyoming        Washakie    968        Wyoming        Washakie      26
           2021-08-12        Wyoming          Weston    701        Wyoming          Weston       6

[1898184 rows x 6 columns]

### Join Dataset and Make STATES dict

In [13]:
unique_states = list(abbrev_to_state_meta.keys())
len(unique_states)

52

In [14]:
STATES = {}

for state in unique_states:
    ## Reset the state's datafranes for pd.merge
    df_cases_temp = df_cases_ts.loc[state].reset_index()
    df_deaths_temp = df_deaths_ts.loc[state].reset_index()

    ## Merge using date and Admin2/county
    df_merged = pd.merge(df_cases_temp, df_deaths_temp, on=['Date','Admin2'])
    df_merged_ts = df_merged.set_index("Date").resample('D').sum()

    df_merged_ts.columns = [f"{c}-{state}" for c in df_merged_ts.columns]    
    
    STATES[state] = df_merged_ts.copy()

STATES.keys()

dict_keys(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'])

In [15]:
## Saving CSVs
folder = os.path.join(fpath_clean,'state_data/')
os.makedirs(folder,exist_ok=True)
os.listdir(folder)

[]

In [16]:
raise Exception('Stop here')

Exception: Stop here

# OLD BELOW

In [ ]:
## Prep Hospital Data

df1 = get_hospital_data()
df1['date'] = pd.to_datetime(df1['date'])

## Remnaming state columsn to match
df1 = df1.rename({'state':'State_Code'},axis=1)
df1['Province_State'] = df1['State_Code'].map(abbrev_to_state_map)
df1 = df1[df1['Province_State'].isin(state_abbrevs['State'])]
df1 = df1.sort_values(['Province_State','date'])
df1 = df1.drop_duplicates(keep='first')
df1

### Making `df_hospitals`

In [ ]:
## needed cols (utilization )
inpatient_bed_util_cols = [c for c in df1.columns if 'inpatient_beds_utilization' in c]
adult_icu_util_cols = [c for c in df1.columns if 'adult_icu_bed_utilization'in c]
KEEP_COLS = ['date','Province_State','State_Code',*inpatient_bed_util_cols,*adult_icu_util_cols]

In [ ]:
df1[df1.duplicated(keep=False)]

In [ ]:
## Making df_hospitals
df_hospitals = df1[KEEP_COLS].copy()
df_hospitals = df_hospitals.set_index(['State_Code','date']).sort_index()
df_hospitals#.loc['MD',['inpatient_beds_utilization']].plot()

## Comparing the 3 tables

In [ ]:
df_hospitals

In [ ]:
df_deaths_ts

In [ ]:
df_cases_ts

In [ ]:
# raise Exception('Stop here!')

## Join Dataset and Make STATES dict
- Note: try to join hospital data here too??

In [ ]:
unique_states = np.unique(df_hospitals.index.get_level_values(0))
len(unique_states)

### Test individual state before making loop

In [ ]:
## ESSENTIAL BLOCK FOR LOOP
state = 'MD'
df_cases_temp = df_cases_ts.loc[state].sort_index().resample("D").sum().diff().fillna(0)
df_deaths_temp = df_deaths_ts.loc[state].sort_index().resample("D").sum().diff().fillna(0)
df_hospital_temp = df_hospitals.loc[state].drop(columns='Province_State').sort_index().resample("D").asfreq().ffill().fillna(0)

In [ ]:
df_hospital_temp

In [ ]:
df_cases_temp.plot()
df_deaths_temp.plot()
df_hospital_temp.plot()

In [ ]:

display(df_cases_temp.head(),df_deaths_temp.head(),df_hospital_temp.head())

### Attempting Various versions of joins

In [ ]:
df_state = pd.concat([df_cases_temp,df_deaths_temp,df_hospital_temp],axis=1).fillna(0)#.loc['03-2020':]
df_state

> ABOVE WAS TEST FOR LOOP BELOW

# FULL LOOP

In [ ]:
STATES = {}
DATA_FOLDER = "./data/"

for state in unique_states:
    df_cases_temp = df_cases_ts.loc[state].sort_index().resample("D").sum().diff().fillna(0)
    df_deaths_temp = df_deaths_ts.loc[state].sort_index().resample("D").sum().diff().fillna(0)
    df_hospital_temp = df_hospitals.loc[state].drop(columns='Province_State').sort_index().resample("D").asfreq().ffill().fillna(0)
    
    df_state = pd.concat([df_cases_temp,df_deaths_temp,df_hospital_temp],axis=1).fillna(0)#.loc['03-2020':]
    df_state.to_csv(f"{DATA_FOLDER}combined_data_{state}.csv.gz",compression='gzip')   
    STATES[state] = df_state.copy()

STATES.keys()

In [ ]:
STATES['MD']

## Saving Data

In [ ]:
import joblib
joblib.dump(STATES,'data/STATE_DICT.joblib')

In [ ]:
STATES_LOADED = joblib.load('data/STATE_DICT.joblib')
STATES_LOADED['TX']

In [ ]:
STATES_LOADED['TX'].plot()